## Adaptive Grid Sizing

CDP allows kernels to decide at runtime how much parallelism to spawn based on actual data.

In [ ]:
%%writefile adaptive_grid.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Worker kernel - processes items in range
__global__ void processRange(int* data, int start, int end, int* output) {
    int idx = start + blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < end) {
        output[idx] = data[idx] * data[idx];  // Square each element
    }
}

// Coordinator kernel - adapts parallelism to workload
__global__ void adaptiveProcess(int* data, int* counts, int numRegions, int* output) {
    int region = blockIdx.x;
    if (region >= numRegions) return;
    
    int count = counts[region];
    int start = 0;
    for (int i = 0; i < region; i++) start += counts[i];
    int end = start + count;
    
    // Adapt grid size to actual workload
    if (count > 0) {
        int threadsPerBlock = 128;
        int blocks = (count + threadsPerBlock - 1) / threadsPerBlock;
        
        printf("Region %d: %d items -> launching %d blocks\n", region, count, blocks);
        processRange<<<blocks, threadsPerBlock>>>(data, start, end, output);
    }
}

int main() {
    // Variable-sized regions (simulating irregular workload)
    int h_counts[] = {100, 5, 500, 20, 1000};
    int numRegions = 5;
    int totalItems = 0;
    for (int i = 0; i < numRegions; i++) totalItems += h_counts[i];
    
    printf("Total items: %d across %d regions\n", totalItems, numRegions);
    
    int *d_data, *d_counts, *d_output;
    cudaMalloc(&d_data, totalItems * sizeof(int));
    cudaMalloc(&d_counts, numRegions * sizeof(int));
    cudaMalloc(&d_output, totalItems * sizeof(int));
    
    // Initialize data
    int* h_data = new int[totalItems];
    for (int i = 0; i < totalItems; i++) h_data[i] = i;
    cudaMemcpy(d_data, h_data, totalItems * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_counts, h_counts, numRegions * sizeof(int), cudaMemcpyHostToDevice);
    
    // Launch adaptive coordinator
    adaptiveProcess<<<numRegions, 1>>>(d_data, d_counts, numRegions, d_output);
    cudaDeviceSynchronize();
    
    // Verify results
    int* h_output = new int[totalItems];
    cudaMemcpy(h_output, d_output, totalItems * sizeof(int), cudaMemcpyDeviceToHost);
    printf("\nSample results: output[0]=%d, output[100]=%d, output[600]=%d\n",
           h_output[0], h_output[100], h_output[600]);
    
    delete[] h_data;
    delete[] h_output;
    cudaFree(d_data);
    cudaFree(d_counts);
    cudaFree(d_output);
    
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt adaptive_grid.cu -o adaptive_grid && ./adaptive_grid

## Quadtree Construction with CDP

Quadtrees adaptively subdivide 2D space based on point density - perfect for CDP.

In [ ]:
%%writefile cdp_quadtree.cu
#include <stdio.h>
#include <cuda_runtime.h>

#define MAX_POINTS_PER_NODE 4
#define MAX_DEPTH 8

struct Point {
    float x, y;
};

struct BoundingBox {
    float minX, minY, maxX, maxY;
    
    __device__ bool contains(Point p) {
        return p.x >= minX && p.x <= maxX && p.y >= minY && p.y <= maxY;
    }
    
    __device__ BoundingBox quadrant(int q) {
        float midX = (minX + maxX) / 2;
        float midY = (minY + maxY) / 2;
        switch (q) {
            case 0: return {minX, midY, midX, maxY};  // NW
            case 1: return {midX, midY, maxX, maxY};  // NE
            case 2: return {minX, minY, midX, midY};  // SW
            case 3: return {midX, minY, maxX, midY};  // SE
        }
        return *this;
    }
};

// Count points in bounding box
__device__ int countPointsInBox(Point* points, int n, BoundingBox box) {
    int count = 0;
    for (int i = 0; i < n; i++) {
        if (box.contains(points[i])) count++;
    }
    return count;
}

// Recursive quadtree construction
__global__ void buildQuadtree(Point* points, int n, BoundingBox box, int depth, int* nodeCount) {
    int count = countPointsInBox(points, n, box);
    
    if (count == 0 || depth >= MAX_DEPTH) {
        return;
    }
    
    // Count this node
    int nodeId = atomicAdd(nodeCount, 1);
    
    if (count <= MAX_POINTS_PER_NODE) {
        // Leaf node - no further subdivision
        printf("Leaf node %d at depth %d: %d points\n", nodeId, depth, count);
        return;
    }
    
    // Internal node - subdivide
    printf("Internal node %d at depth %d: %d points -> subdividing\n", nodeId, depth, count);
    
    // Launch children for each quadrant
    for (int q = 0; q < 4; q++) {
        BoundingBox childBox = box.quadrant(q);
        buildQuadtree<<<1, 1>>>(points, n, childBox, depth + 1, nodeCount);
    }
    cudaDeviceSynchronize();
}

int main() {
    const int N = 100;
    Point h_points[N];
    
    // Generate clustered points
    srand(42);
    for (int i = 0; i < N/2; i++) {
        // Cluster 1: upper-right
        h_points[i] = {0.7f + 0.3f * rand() / RAND_MAX, 0.7f + 0.3f * rand() / RAND_MAX};
    }
    for (int i = N/2; i < N; i++) {
        // Cluster 2: lower-left
        h_points[i] = {0.0f + 0.3f * rand() / RAND_MAX, 0.0f + 0.3f * rand() / RAND_MAX};
    }
    
    Point* d_points;
    int* d_nodeCount;
    cudaMalloc(&d_points, N * sizeof(Point));
    cudaMalloc(&d_nodeCount, sizeof(int));
    
    cudaMemcpy(d_points, h_points, N * sizeof(Point), cudaMemcpyHostToDevice);
    cudaMemset(d_nodeCount, 0, sizeof(int));
    
    BoundingBox rootBox = {0, 0, 1, 1};
    buildQuadtree<<<1, 1>>>(d_points, N, rootBox, 0, d_nodeCount);
    cudaDeviceSynchronize();
    
    int h_nodeCount;
    cudaMemcpy(&h_nodeCount, d_nodeCount, sizeof(int), cudaMemcpyDeviceToHost);
    printf("\nTotal nodes created: %d\n", h_nodeCount);
    
    cudaFree(d_points);
    cudaFree(d_nodeCount);
    
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt cdp_quadtree.cu -o cdp_quadtree && ./cdp_quadtree

## When CDP Shines vs. When to Avoid

### Good Use Cases ✅
- Recursive divide-and-conquer
- Irregular/adaptive workloads
- Tree/graph traversal
- Workload discovered at runtime

### Avoid When ❌
- Regular, predictable parallelism
- Very deep recursion (>16 levels)
- Launching many tiny kernels
- CPU coordination is acceptable

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to reinforce your understanding of adaptive algorithms with CDP:

**Exercise 1: Adaptive Sparse Matrix Processing**
Process only non-zero regions of a sparse matrix using CDP. The coordinator kernel should analyze density and spawn workers only for dense regions.

**Exercise 2: Dynamic Load Balancing**
Implement a work-stealing pattern where idle regions can help process work from overloaded regions using adaptive kernel launches.

**Exercise 3: Hierarchical Octree Traversal**
Build a 3D octree and use CDP to adaptively process only occupied nodes, spawning 8 child kernels for non-leaf nodes.

In [ ]:
%%writefile adaptive_exercises.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>

// ============================================================
// Exercise 1: Adaptive Sparse Matrix Processing with CDP
// ============================================================

#define BLOCK_SIZE 16
#define DENSITY_THRESHOLD 0.3f

__global__ void processBlock(float* matrix, int startRow, int startCol, 
                             int blockRows, int blockCols, int matrixWidth,
                             float* output) {
    int localRow = threadIdx.y;
    int localCol = threadIdx.x;
    
    if (localRow < blockRows && localCol < blockCols) {
        int globalRow = startRow + localRow;
        int globalCol = startCol + localCol;
        int idx = globalRow * matrixWidth + globalCol;
        
        // Process: square non-zero elements
        if (matrix[idx] != 0.0f) {
            output[idx] = matrix[idx] * matrix[idx];
        }
    }
}

__global__ void sparseMatrixCoordinator(float* matrix, int rows, int cols, 
                                        float* output, int* blockCounts) {
    int blockRow = blockIdx.y;
    int blockCol = blockIdx.x;
    
    int startRow = blockRow * BLOCK_SIZE;
    int startCol = blockCol * BLOCK_SIZE;
    int blockRows = min(BLOCK_SIZE, rows - startRow);
    int blockCols = min(BLOCK_SIZE, cols - startCol);
    
    // Count non-zeros in this block
    int nonZeros = 0;
    for (int r = 0; r < blockRows; r++) {
        for (int c = 0; c < blockCols; c++) {
            int idx = (startRow + r) * cols + (startCol + c);
            if (matrix[idx] != 0.0f) nonZeros++;
        }
    }
    
    float density = (float)nonZeros / (blockRows * blockCols);
    int blockId = blockRow * gridDim.x + blockCol;
    blockCounts[blockId] = nonZeros;
    
    // Only launch child kernel if density exceeds threshold
    if (density >= DENSITY_THRESHOLD) {
        dim3 threads(BLOCK_SIZE, BLOCK_SIZE);
        processBlock<<<1, threads>>>(matrix, startRow, startCol, 
                                     blockRows, blockCols, cols, output);
        printf("Block[%d,%d]: density=%.2f, launching worker\n", 
               blockRow, blockCol, density);
    } else {
        printf("Block[%d,%d]: density=%.2f, skipping (sparse)\n", 
               blockRow, blockCol, density);
    }
}

void exercise1_sparse_matrix() {
    printf("=== Exercise 1: Adaptive Sparse Matrix Processing ===\n");
    
    const int ROWS = 32, COLS = 32;
    float* h_matrix = (float*)calloc(ROWS * COLS, sizeof(float));
    float* h_output = (float*)calloc(ROWS * COLS, sizeof(float));
    
    // Create sparse matrix with dense regions
    // Dense region in top-left quadrant
    for (int r = 0; r < 16; r++) {
        for (int c = 0; c < 16; c++) {
            if ((r + c) % 2 == 0) h_matrix[r * COLS + c] = (float)(r + c + 1);
        }
    }
    // Sparse bottom-right
    h_matrix[20 * COLS + 20] = 5.0f;
    
    float *d_matrix, *d_output;
    int* d_blockCounts;
    int numBlocksX = (COLS + BLOCK_SIZE - 1) / BLOCK_SIZE;
    int numBlocksY = (ROWS + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    cudaMalloc(&d_matrix, ROWS * COLS * sizeof(float));
    cudaMalloc(&d_output, ROWS * COLS * sizeof(float));
    cudaMalloc(&d_blockCounts, numBlocksX * numBlocksY * sizeof(int));
    cudaMemcpy(d_matrix, h_matrix, ROWS * COLS * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemset(d_output, 0, ROWS * COLS * sizeof(float));
    
    dim3 grid(numBlocksX, numBlocksY);
    sparseMatrixCoordinator<<<grid, 1>>>(d_matrix, ROWS, COLS, d_output, d_blockCounts);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_output, d_output, ROWS * COLS * sizeof(float), cudaMemcpyDeviceToHost);
    
    // Verify some outputs
    printf("Sample outputs: [0,0]=%.0f (expected %.0f), [1,1]=%.0f (expected %.0f)\n",
           h_output[0], h_matrix[0] * h_matrix[0],
           h_output[COLS + 1], h_matrix[COLS + 1] * h_matrix[COLS + 1]);
    
    free(h_matrix); free(h_output);
    cudaFree(d_matrix); cudaFree(d_output); cudaFree(d_blockCounts);
    printf("\n");
}

// ============================================================
// Exercise 2: Dynamic Load Balancing with CDP
// ============================================================

__device__ int workQueue[1024];
__device__ int queueHead = 0;
__device__ int queueTail = 0;

__global__ void workerKernel(int* data, int workId, int workSize, int* results) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < workSize) {
        // Simulate work with variable complexity
        int sum = 0;
        for (int i = 0; i < data[workId] * 10; i++) {
            sum += i;
        }
        atomicAdd(&results[workId], sum);
    }
}

__global__ void loadBalancer(int* workloads, int numWorks, int* results) {
    int workerId = blockIdx.x;
    int myWork = workerId;
    
    if (myWork >= numWorks) return;
    
    int workSize = workloads[myWork];
    printf("Worker %d: processing work %d (size=%d)\n", workerId, myWork, workSize);
    
    // Adapt parallelism based on workload size
    int threads = min(256, workSize);
    int blocks = (workSize + threads - 1) / threads;
    blocks = max(1, blocks);
    
    workerKernel<<<blocks, threads>>>(workloads, myWork, workSize, results);
}

void exercise2_load_balancing() {
    printf("=== Exercise 2: Dynamic Load Balancing ===\n");
    
    // Variable workloads
    int h_workloads[] = {10, 500, 20, 1000, 5, 800, 15, 300};
    int numWorks = 8;
    
    int* d_workloads;
    int* d_results;
    int h_results[8] = {0};
    
    cudaMalloc(&d_workloads, numWorks * sizeof(int));
    cudaMalloc(&d_results, numWorks * sizeof(int));
    cudaMemcpy(d_workloads, h_workloads, numWorks * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(d_results, 0, numWorks * sizeof(int));
    
    loadBalancer<<<numWorks, 1>>>(d_workloads, numWorks, d_results);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_results, d_results, numWorks * sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("Work processing completed.\n\n");
    
    cudaFree(d_workloads);
    cudaFree(d_results);
}

// ============================================================
// Exercise 3: Hierarchical Octree Traversal with CDP
// ============================================================

struct OctreeNode {
    int occupied;     // 8 bits, one per child
    int isLeaf;
    int childBase;    // Index of first child (8 consecutive)
    float data;
};

__global__ void processOctreeNode(OctreeNode* octree, int nodeIdx, 
                                  float* results, int* processedCount) {
    OctreeNode node = octree[nodeIdx];
    
    if (node.isLeaf) {
        // Process leaf node
        int idx = atomicAdd(processedCount, 1);
        results[idx] = node.data;
        printf("Leaf node %d: data=%.1f\n", nodeIdx, node.data);
        return;
    }
    
    // Count occupied children
    int numChildren = __popc(node.occupied);
    
    if (numChildren > 0) {
        printf("Node %d: %d occupied children, spawning workers\n", nodeIdx, numChildren);
        
        // Spawn child kernels for occupied nodes
        for (int i = 0; i < 8; i++) {
            if (node.occupied & (1 << i)) {
                int childIdx = node.childBase + i;
                processOctreeNode<<<1, 1>>>(octree, childIdx, results, processedCount);
            }
        }
        cudaDeviceSynchronize();
    }
}

void exercise3_octree() {
    printf("=== Exercise 3: Hierarchical Octree Traversal ===\n");
    
    // Build simple octree:
    // Root (node 0) has 3 occupied children
    // Some children are leaves, some have grandchildren
    
    OctreeNode h_octree[12] = {
        {0b00000111, 0, 1, 0.0f},   // Node 0: root, children at 1,2,3
        {0, 1, 0, 1.0f},            // Node 1: leaf
        {0b00000011, 0, 4, 0.0f},   // Node 2: 2 children at 4,5
        {0, 1, 0, 3.0f},            // Node 3: leaf
        {0, 1, 0, 4.0f},            // Node 4: leaf
        {0, 1, 0, 5.0f},            // Node 5: leaf
    };
    
    OctreeNode* d_octree;
    float* d_results;
    int* d_count;
    float h_results[10] = {0};
    int h_count = 0;
    
    cudaMalloc(&d_octree, 12 * sizeof(OctreeNode));
    cudaMalloc(&d_results, 10 * sizeof(float));
    cudaMalloc(&d_count, sizeof(int));
    
    cudaMemcpy(d_octree, h_octree, 12 * sizeof(OctreeNode), cudaMemcpyHostToDevice);
    cudaMemset(d_results, 0, 10 * sizeof(float));
    cudaMemset(d_count, 0, sizeof(int));
    
    processOctreeNode<<<1, 1>>>(d_octree, 0, d_results, d_count);
    cudaDeviceSynchronize();
    
    cudaMemcpy(&h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_results, d_results, h_count * sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("Processed %d leaf nodes with values: ", h_count);
    for (int i = 0; i < h_count; i++) printf("%.0f ", h_results[i]);
    printf("\n\n");
    
    cudaFree(d_octree);
    cudaFree(d_results);
    cudaFree(d_count);
}

int main() {
    printf("CDP Adaptive Algorithms Exercises\n");
    printf("==================================\n\n");
    
    exercise1_sparse_matrix();
    exercise2_load_balancing();
    exercise3_octree();
    
    printf("All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -lcudadevrt -o adaptive_exercises adaptive_exercises.cu && ./adaptive_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises explore similar adaptive concepts using Python. Note that Numba CUDA does not support dynamic parallelism directly, so these focus on alternative approaches.

**Exercise A:** Implement a sparse matrix processor in Numba that uses a pre-computed density map to skip empty regions.

**Exercise B:** Create an adaptive workload distributor using Python multiprocessing combined with Numba CUDA kernels for the compute-intensive portions.

## Key Takeaways

1. **Adaptive parallelism** - spawn work based on actual data
2. **Hierarchical structures** - quadtree/octree natural fit
3. **Irregular workloads** - regions with variable sizes
4. **Trade-offs** - launch overhead vs. CPU roundtrip

## Next: Day 4 - CDP Optimization & Best Practices